In [1]:
#pip install selenium

In [2]:
#!pip install selenium webdriver_manager

In [29]:
import pandas as pd

import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import re

In [75]:
# rank: 순위
# song_name: 곡 제목
# artist: 가수명
# likes: 좋아요 수
# 앨범ID: album_id
# 가사ID: song_id
def make_info_list(trs, year):
    all_list = []
    for tr in trs:
        rank = tr.find_element(By.CSS_SELECTOR, ".right_none > span").text.strip()
        song_name = tr.find_element(By.CSS_SELECTOR, "#chartListObj > tr .rank01").text.strip()
        artist = tr.find_element(By.CSS_SELECTOR, ".rank02 a").text.strip()
        likes = tr.find_element(By.CSS_SELECTOR, ".t_left .cnt").text.strip()
        # 앨범사진 속성값 가져오기
        image = tr.find_element(By.CSS_SELECTOR, ".bg_album_frame")
        image2 = image.get_attribute("onclick")
        match = re.search(r"\d+", image2)
        album_id = match.group()
        #가사 속성값 가져오기
        text = tr.find_element(By.CSS_SELECTOR, "button.btn_icon.add")
        text2 = text.get_attribute("onclick")
        match2 = re.search(r"(\d+)", text2)
        song_id = match2.group()

        one_list = [year, rank, song_name, artist, likes, album_id, song_id]
        all_list.append(one_list)

    return all_list


In [108]:
service = Service(ChromeDriverManager().install())
d = webdriver.Chrome(service = service)

try:
    d.get("https://www.melon.com/index.htm")
    time.sleep(3)

    # 멜론 차트 클릭하기
    chart = d.find_element(By.CSS_SELECTOR, ".nth1") 
    ac = ActionChains(d)
    ac.move_to_element(chart)
    ac.click()
    ac.pause(3) #기다리기
    ac.perform()

    #차트 파인더 클릭하기
    finder = d.find_element(By.CSS_SELECTOR, ".chart_finder") 
    ac.reset_actions() #액션을 리셋
    ac.move_to_element(finder)
    ac.click()
    ac.pause(3) #기다리기
    ac.perform()

    #연도차트 클릭하기
    finder_year = d.find_element(By.CSS_SELECTOR, ".tab03") 
    ac.reset_actions() #액션을 리셋
    ac.move_to_element(finder_year)
    ac.click()
    ac.pause(3) #기다리기
    ac.perform()

    #연대 선택 클릭하기
    years = d.find_elements(By.CSS_SELECTOR, ".list_value> ul > li") 

    # 전체 리스트
    all_list = []
    for ten_year in years[:2]:
        ac.reset_actions() #액션을 리셋
        ac.move_to_element(ten_year)
        ac.click()
        ac.pause(3) #기다리기
        ac.perform()
    
        #연도선택 클릭하기
        year_list = d.find_elements(By.CSS_SELECTOR, ".box_chic.nth2> .list_value> ul > li") 
        year_list_text = d.find_element(By.CSS_SELECTOR, ".box_chic.nth2> .list_value").text.strip()
        year_list_text = year_list_text.split()
    
        for i, year in enumerate(year_list):
            year_list = []
            
            ac.reset_actions() #액션을 리셋
            ac.move_to_element(year)
            ac.click()
            ac.pause(3) #기다리기
            ac.perform()
        
            #장르선택 클릭하기
            style = d.find_elements(By.CSS_SELECTOR, ".box_chic.last> .list_value> ul > li") 
            ac.reset_actions() #액션을 리셋
            ac.move_to_element(style[1])
            ac.click()
            ac.pause(3) #기다리기
            ac.perform()
        
            #검색 클릭하기
            click = d.find_element(By.CSS_SELECTOR, ".btn_b26") 
            ac.reset_actions() #액션을 리셋
            ac.move_to_element(click)
            ac.click()
            ac.pause(10) #기다리기
            ac.perform()
    
            year_text = year_list_text[i]
    
            print(year_text)
            
            
        
            # 1~50위
            trs = d.find_elements(By.CSS_SELECTOR, "#chartListObj .lst50") #1-50
            
            lst1 = make_info_list(trs, year_text)
        
            # 51~100위
            # 다음 페이지 이동하기
            next = d.find_element(By.CSS_SELECTOR, ".page_num a") #51-100위
            ac.reset_actions() #액션을 리셋
            ac.move_to_element(next)
            ac.click()
            ac.pause(5) #기다리기
            ac.perform()    
            
            
            trs = d.find_elements(By.CSS_SELECTOR, "#chartListObj .lst100") #51-100
            lst2 = make_info_list(trs, year_text)
        
            year_list = lst1 + lst2

            
            all_list += year_list
            
        
       

except Exception as e:
    print(e)

finally:
    d.close()
    d.quit()

2022년
2021년
2020년
2019년
2018년
2017년
2016년
2015년
2014년
2013년
2012년
2011년
2010년


In [109]:
len(all_list)

1300

In [110]:
columns = ['year', 'rank', 'song_name', 'artist', 'likes', 'album_id', 'song_id']

result_df = pd.DataFrame(all_list, columns=columns)

In [111]:
result_df

,year,rank,song_name,artist,likes,album_id,song_id
0,2022년,1,LOVE DIVE,IVE (아이브),"242,801",10909179,34847378
1,2022년,2,TOMBOY,(여자)아이들,"241,400",10890384,34754292
2,2022년,3,취중고백,김민석 (멜로망스),"173,287",10816959,34431086
3,2022년,4,사랑인가 봐,멜로망스,"194,126",10871162,34657844
4,2022년,5,사랑은 늘 도망가,임영웅,"202,918",10735654,34061322
...,...,...,...,...,...,...,...
1295,2010년,96,Hip Song,비,"9,490",841954,2626611
1296,2010년,97,가로수 그늘 아래 서면,장재인(JEIN),"16,734",1053361,2989873
1297,2010년,98,이별이 온다,에이트,"6,548",882152,2673348
1298,2010년,99,19금 부치지 못한 편지,DJ DOC,"16,031",968252,2783256


In [112]:
# csv 파일 생성
result_df.to_csv('순위.csv', encoding='utf-8')

In [113]:
# csv 파일 읽기
pd.read_csv('순위.csv', encoding='utf-8')

,Unnamed: 0,year,rank,song_name,artist,likes,album_id,song_id
0,0,2022년,1,LOVE DIVE,IVE (아이브),"242,801",10909179,34847378
1,1,2022년,2,TOMBOY,(여자)아이들,"241,400",10890384,34754292
2,2,2022년,3,취중고백,김민석 (멜로망스),"173,287",10816959,34431086
3,3,2022년,4,사랑인가 봐,멜로망스,"194,126",10871162,34657844
4,4,2022년,5,사랑은 늘 도망가,임영웅,"202,918",10735654,34061322
...,...,...,...,...,...,...,...,...
1295,1295,2010년,96,Hip Song,비,"9,490",841954,2626611
1296,1296,2010년,97,가로수 그늘 아래 서면,장재인(JEIN),"16,734",1053361,2989873
1297,1297,2010년,98,이별이 온다,에이트,"6,548",882152,2673348
1298,1298,2010년,99,19금 부치지 못한 편지,DJ DOC,"16,031",968252,2783256
